In [3]:
# 1 首先找到出发站点start_station所在的地铁是几号线即subway_name
# 2 找到该线路上离出发站点最近的站点next_station
# 3 判断next_station是否与目标站点end_station一致 若一致则直接返回 [start_station] + [next_station]
# 4 若不一致 则将 [start_station] + [next_station] 存放在已访问列表 visited 中
# 5 然后继续判断next_station有几条地铁线路经过 
# 6 选出离next_station最近的sub_next_station 判断sub_next_station是否已经存在于visited中
# 7 若存在直接continue 若不存在 如上步骤3456进行递归直到找到目标站点end_station为止

### BFS station to station

In [4]:
import urllib.request as ur
import json
import csv

In [5]:
#  直接调用高德的api 获取上海地铁线路 站点 经纬度信息
#  若需要查看其他城市 将url1中的(3100,shanghai)修改成对应城市的id和拼音
url1 = "http://map.amap.com/service/subway?_1469083453978&srhdata=3100_drw_shanghai.json"

In [6]:
# 获取高德api网页信息
def get_response(url):
    html = ur.urlopen(url)
    hjson = json.loads(html.read().decode("utf-8"))
    return hjson

In [7]:
# 解析json数据
def paras(url):
    hjson = get_response(url)
    info = hjson['l']
    st_dict = dict()
    st_location = dict()
    for i in info :
        kn = i['kn']   # 地铁线路名
        st = i['st']   # 站点信息
        st_list = []   # 用来储存站点名的list
        for s in st:
            st_name = s['n']
            st_list.append(st_name)
            st_long,st_lat = s['sl'].split(',')
            st_location[st_name] = (float(st_long),float(st_lat))
        st_dict[kn] = st_list
    return st_dict,st_location

In [8]:
station_info,st_location = paras(url1)    

In [9]:
# 地铁线路经过站点信息
station_info

{'地铁1号线': ['莘庄',
  '外环路',
  '莲花路',
  '锦江乐园',
  '上海南站',
  '漕宝路',
  '上海体育馆',
  '徐家汇',
  '衡山路',
  '常熟路',
  '陕西南路',
  '黄陂南路',
  '人民广场',
  '新闸路',
  '汉中路',
  '上海火车站',
  '中山北路',
  '延长路',
  '上海马戏城',
  '汶水路',
  '彭浦新村',
  '共康路',
  '通河新村',
  '呼兰路',
  '共富新村',
  '宝安公路',
  '友谊西路',
  '富锦路'],
 '地铁2号线': ['浦东国际机场',
  '海天三路',
  '远东大道',
  '凌空路',
  '川沙',
  '华夏东路',
  '创新中路',
  '唐镇',
  '广兰路',
  '金科路',
  '张江高科',
  '龙阳路',
  '世纪公园',
  '上海科技馆',
  '世纪大道',
  '东昌路',
  '陆家嘴',
  '南京东路',
  '人民广场',
  '南京西路',
  '静安寺',
  '江苏路',
  '中山公园',
  '娄山关路',
  '威宁路',
  '北新泾',
  '淞虹路',
  '虹桥2号航站楼',
  '虹桥火车站',
  '徐泾东'],
 '地铁3号线': ['江杨北路',
  '铁力路',
  '友谊路',
  '宝杨路',
  '水产路',
  '淞滨路',
  '张华浜',
  '淞发路',
  '长江南路',
  '殷高西路',
  '江湾镇',
  '大柏树',
  '赤峰路',
  '虹口足球场',
  '东宝兴路',
  '宝山路',
  '上海火车站',
  '中潭路',
  '镇坪路',
  '曹杨路',
  '金沙江路',
  '中山公园',
  '延安西路',
  '虹桥路',
  '宜山路',
  '漕溪路',
  '龙漕路',
  '石龙路',
  '上海南站'],
 '地铁4号线': ['宜山路',
  '虹桥路',
  '延安西路',
  '中山公园',
  '金沙江路',
  '曹杨路',
  '镇坪路',
  '中潭路',
  '上海火车站',
  '宝山路',
  '海伦路',
  '临平路',
  '大连路',
  '杨树浦路

In [10]:
# 各站点经纬度
st_location

{'莘庄': (121.385373, 31.111152),
 '外环路': (121.39302, 31.120899),
 '莲花路': (121.40291, 31.1309),
 '锦江乐园': (121.414107, 31.142217),
 '上海南站': (121.430041, 31.154579),
 '漕宝路': (121.433143, 31.168344),
 '上海体育馆': (121.437423, 31.182813),
 '徐家汇': (121.436837, 31.195338),
 '衡山路': (121.446424, 31.204528),
 '常熟路': (121.449141, 31.213524),
 '陕西南路': (121.458744, 31.21515),
 '黄陂南路': (121.473306, 31.222745),
 '人民广场': (121.475137, 31.232781),
 '新闸路': (121.468151, 31.238373),
 '汉中路': (121.458699, 31.241883),
 '上海火车站': (121.457939, 31.249632),
 '中山北路': (121.459204, 31.258891),
 '延长路': (121.455329, 31.271675),
 '上海马戏城': (121.452023, 31.279895),
 '汶水路': (121.450251, 31.292556),
 '彭浦新村': (121.448642, 31.306604),
 '共康路': (121.447063, 31.318936),
 '通河新村': (121.441546, 31.33113),
 '呼兰路': (121.437711, 31.339703),
 '共富新村': (121.434063, 31.355082),
 '宝安公路': (121.430914, 31.369555),
 '友谊西路': (121.427953, 31.381296),
 '富锦路': (121.424661, 31.39226),
 '浦东国际机场': (121.805591, 31.150958),
 '海天三路': (121.796878, 31.168459

In [11]:
# 获取某站点可直接到达的下一站
def get_next_station(start):
    nearest_st = []
    for k,v in station_info.items():
        if start in v:
            station_index = v.index(start)
            if station_index != 0:
                nearest_st.append(v[station_index-1])
            if station_index != (len(v)-1):
                nearest_st.append(v[station_index+1])
    return nearest_st

In [12]:
get_next_station("人民广场")

['黄陂南路', '新闸路', '南京东路', '南京西路', '大世界', '曲阜路']

In [13]:
def bfs_to_station(start, end):
    path_list = [[start]]
    visited = [start]

    while path_list:
        path = path_list.pop(0)
        station = path[-1]
        next_station_list = get_next_station(station)
        for next_station in next_station_list:
            if next_station in visited: continue
            if next_station == end:
                return path + [next_station]
            visited.append(next_station)
            path_list.append(path + [next_station])

In [14]:
def search(start,end):
    path_list = bfs_to_station(start,end)
    return "->".join(path_list)

In [15]:
search("徐家汇","陆家嘴")

'徐家汇->衡山路->常熟路->陕西南路->黄陂南路->人民广场->南京东路->陆家嘴'

In [16]:
search("同济大学","人民广场")

'同济大学->四平路->曲阳路->虹口足球场->西藏北路->中兴路->曲阜路->人民广场'

In [17]:
search("虹口足球场","莘庄")

'虹口足球场->东宝兴路->宝山路->上海火车站->汉中路->南京西路->静安寺->江苏路->交通大学->徐家汇->上海体育馆->漕宝路->上海南站->锦江乐园->莲花路->外环路->莘庄'

In [18]:
search("虹漕路","张江高科")

'虹漕路->桂林公园->漕宝路->龙漕路->龙华->云锦路->龙耀路->东方体育中心->三林->三林东->浦三路->御桥->罗山路->华夏中路->龙阳路->张江高科'

### station to station with shortest path

In [19]:
st_location

{'莘庄': (121.385373, 31.111152),
 '外环路': (121.39302, 31.120899),
 '莲花路': (121.40291, 31.1309),
 '锦江乐园': (121.414107, 31.142217),
 '上海南站': (121.430041, 31.154579),
 '漕宝路': (121.433143, 31.168344),
 '上海体育馆': (121.437423, 31.182813),
 '徐家汇': (121.436837, 31.195338),
 '衡山路': (121.446424, 31.204528),
 '常熟路': (121.449141, 31.213524),
 '陕西南路': (121.458744, 31.21515),
 '黄陂南路': (121.473306, 31.222745),
 '人民广场': (121.475137, 31.232781),
 '新闸路': (121.468151, 31.238373),
 '汉中路': (121.458699, 31.241883),
 '上海火车站': (121.457939, 31.249632),
 '中山北路': (121.459204, 31.258891),
 '延长路': (121.455329, 31.271675),
 '上海马戏城': (121.452023, 31.279895),
 '汶水路': (121.450251, 31.292556),
 '彭浦新村': (121.448642, 31.306604),
 '共康路': (121.447063, 31.318936),
 '通河新村': (121.441546, 31.33113),
 '呼兰路': (121.437711, 31.339703),
 '共富新村': (121.434063, 31.355082),
 '宝安公路': (121.430914, 31.369555),
 '友谊西路': (121.427953, 31.381296),
 '富锦路': (121.424661, 31.39226),
 '浦东国际机场': (121.805591, 31.150958),
 '海天三路': (121.796878, 31.168459

In [20]:
import math

In [21]:
def formula_distance(origin,destination):
    ori_long,ori_lat = origin
    des_long,des_lat = destination
    
    radius = 6371
    
#   已知经纬度可以运用haversine formula 求两点之间的球面距离
#   math.radians 返回一个角度的弧度值  
    dlon = math.radians(des_long - ori_long)
    dlat = math.radians(des_lat - ori_lat)
    
    a = math.sin(dlat/2)**2+math.cos(math.radians(ori_lat))*math.cos(math.radians(des_lat)) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a) , math.sqrt(1-a))
    d = radius * c
    
    return d

In [22]:
def get_st_distance(origin,destination):
    return formula_distance(st_location[origin],st_location[destination])

In [23]:
# 高德地图测距为5.64KM
get_st_distance("徐家汇","人民广场")

5.531773023672401

In [24]:
# 高德地图测距为4.9KM
get_st_distance("同济大学","陆家嘴")  

4.8959535276036155

In [25]:
def bfs_search_by_distance(start,end):
    pathes = [[start]]
    visited = set()
    while pathes:
        path = pathes.pop(0)
        froniter = path[-1]
        
        if froniter in visited:continue
        if froniter == end:
            return path
        
        successsors = get_next_station(froniter)

        for st in successsors:
            if st in path: continue  

            new_path = path + [st]

            pathes.append(new_path)  

        pathes = path_sorted_by_distance(pathes)
        visited.add(froniter)
        if pathes and (end == pathes[0][-1]):
            return pathes[0]

In [26]:
def path_sorted_by_distance(paths):
    def get_path_distance(path):
        distance = 0
        for index,st in enumerate(path[:-1]):
            distance += get_st_distance(path[index],path[index+1])
        return distance
    return sorted(paths,key=get_path_distance)

In [27]:
def search_by_shortest(start,end):
    path_list = bfs_search_by_distance(start,end)
    return "->".join(path_list)

In [28]:
# BFS search
search("人民广场","陆家嘴")

'人民广场->南京东路->陆家嘴'

In [29]:
# 按距离最短search
search_by_shortest("人民广场","陆家嘴")

'人民广场->南京东路->陆家嘴'

In [30]:
# BFS search
search("同济大学","徐家汇")

'同济大学->四平路->邮电新村->海伦路->宝山路->上海火车站->汉中路->南京西路->静安寺->江苏路->交通大学->徐家汇'

In [31]:
# 按距离最短search
search_by_shortest("同济大学","徐家汇")

'同济大学->四平路->邮电新村->海伦路->四川北路->天潼路->南京东路->人民广场->黄陂南路->陕西南路->常熟路->衡山路->徐家汇'

In [32]:
# BFS search
search("莘庄","中山公园")

'莘庄->外环路->莲花路->锦江乐园->上海南站->漕宝路->上海体育馆->徐家汇->交通大学->江苏路->中山公园'

In [33]:
# 按距离最短search
search_by_shortest("莘庄","中山公园")

'莘庄->外环路->莲花路->锦江乐园->上海南站->石龙路->龙漕路->漕溪路->宜山路->虹桥路->延安西路->中山公园'

### minimum transfers between two stations.

In [34]:
# 1 在最短距离的基础上把按距离排序换成按换成次数排序
# 2 需要定义一个函数来计算换成次数

In [35]:
def bfs_search_by_transfers(start,end):
    pathes = [[start]]
    visited = set()
    result = []
    while pathes:
        path = pathes.pop(0)
        froniter = path[-1]
        
        if froniter in visited:continue
        if froniter == end:
            return path
        
        successsors = get_next_station(froniter)
        for st in successsors:
            if st in path: continue  

            new_path = path + [st]

            pathes.append(new_path)  

        pathes = path_sorted_by_transfers(pathes)
        visited.add(froniter)
        
        if pathes and (end == pathes[0][-1]):
            return pathes[0]

In [49]:
bfs_search_by_transfers("静安寺","徐家汇")

['静安寺', '常熟路', '肇嘉浜路', '徐家汇']

In [50]:
def path_sorted_by_transfers(paths):
    def get_path_transfers(path):
        transfers = 0
        for index,st in enumerate(path):
            if index == 0 : continue
            if index == (len(path)-1) : continue
            transfers += get_st_transfers(path[index-1],path[index+1])
        return transfers
    return sorted(paths,key=get_path_transfers)

In [51]:
station_info

{'地铁1号线': ['莘庄',
  '外环路',
  '莲花路',
  '锦江乐园',
  '上海南站',
  '漕宝路',
  '上海体育馆',
  '徐家汇',
  '衡山路',
  '常熟路',
  '陕西南路',
  '黄陂南路',
  '人民广场',
  '新闸路',
  '汉中路',
  '上海火车站',
  '中山北路',
  '延长路',
  '上海马戏城',
  '汶水路',
  '彭浦新村',
  '共康路',
  '通河新村',
  '呼兰路',
  '共富新村',
  '宝安公路',
  '友谊西路',
  '富锦路'],
 '地铁2号线': ['浦东国际机场',
  '海天三路',
  '远东大道',
  '凌空路',
  '川沙',
  '华夏东路',
  '创新中路',
  '唐镇',
  '广兰路',
  '金科路',
  '张江高科',
  '龙阳路',
  '世纪公园',
  '上海科技馆',
  '世纪大道',
  '东昌路',
  '陆家嘴',
  '南京东路',
  '人民广场',
  '南京西路',
  '静安寺',
  '江苏路',
  '中山公园',
  '娄山关路',
  '威宁路',
  '北新泾',
  '淞虹路',
  '虹桥2号航站楼',
  '虹桥火车站',
  '徐泾东'],
 '地铁3号线': ['江杨北路',
  '铁力路',
  '友谊路',
  '宝杨路',
  '水产路',
  '淞滨路',
  '张华浜',
  '淞发路',
  '长江南路',
  '殷高西路',
  '江湾镇',
  '大柏树',
  '赤峰路',
  '虹口足球场',
  '东宝兴路',
  '宝山路',
  '上海火车站',
  '中潭路',
  '镇坪路',
  '曹杨路',
  '金沙江路',
  '中山公园',
  '延安西路',
  '虹桥路',
  '宜山路',
  '漕溪路',
  '龙漕路',
  '石龙路',
  '上海南站'],
 '地铁4号线': ['宜山路',
  '虹桥路',
  '延安西路',
  '中山公园',
  '金沙江路',
  '曹杨路',
  '镇坪路',
  '中潭路',
  '上海火车站',
  '宝山路',
  '海伦路',
  '临平路',
  '大连路',
  '杨树浦路

In [52]:
def get_st_transfers(start,end):
    for k,v in station_info.items():
        if (start in v) and (end in v):return 0
    return 1

In [53]:
bfs_search_by_transfers("静安寺","人民广场")

['静安寺', '南京西路', '人民广场']

In [54]:
def search_by_min_transfers(start,end):
    path_list = bfs_search_by_transfers(start,end)
    return "->".join(path_list)

In [55]:
search_by_min_transfers("静安寺","人民广场")

'静安寺->南京西路->人民广场'

In [72]:
# BFS 搜索路线
search("同济大学","新天地")

'同济大学->四平路->邮电新村->海伦路->宝山路->上海火车站->汉中路->南京西路->陕西南路->新天地'

In [73]:
# 距离最短搜索路线
search_by_shortest("同济大学","新天地")

'同济大学->四平路->邮电新村->海伦路->四川北路->天潼路->南京东路->豫园->老西门->新天地'

In [74]:
# 换乘最少搜索路线
search_by_min_transfers("同济大学","新天地")

'同济大学->四平路->邮电新村->海伦路->四川北路->天潼路->南京东路->豫园->老西门->新天地'

In [75]:
# BFS 搜索路线
search("静安寺","徐家汇")

'静安寺->江苏路->交通大学->徐家汇'

In [76]:
# 距离最短搜索路线
search_by_shortest("静安寺","徐家汇")

'静安寺->常熟路->衡山路->徐家汇'

In [77]:
# 换乘最少搜索路线
search_by_min_transfers("静安寺","徐家汇")

'静安寺->常熟路->肇嘉浜路->徐家汇'

### Combine shortest and minimum transfers

In [ ]:
def combine_dis_trans(start,end):
    pathes = [[start]]
    visited = set()
    while pathes:
        path = pathes.pop(0)
        froniter = path[-1]
        
        if froniter in visited:continue
        if froniter == end:
            return path
        
        successsors = get_next_station(froniter)

        for st in successsors:
            if st in path: continue  

            new_path = path + [st]

            pathes.append(new_path)  
            
#         t = get_transfers(pathes[0])
#         s = get_distance(pathes[0])
#       先用get_transfers 判断出最短换乘 若换乘次数相等  则选择get_istance距离最短的路径
        pathes = sorted(pathes,key=lambda x:(get_transfers(x),get_distance(x)))
        visited.add(froniter)
        if pathes and (end == pathes[0][-1]):
            return pathes[0]

In [ ]:
def get_transfers(path):
    transfers = 0
    for index,st in enumerate(path[:-1]):
        transfers += get_st_transfers(path[index],path[index+1])
    return transfers

In [ ]:
def get_distance(path):
    distance = 0
    for index,st in enumerate(path[:-1]):
        distance += get_st_distance(path[index],path[index+1])
    return distance

In [104]:
def search_combine(start,end):
    path_list = combine_dis_trans(start,end)
    return "->".join(path_list)

In [109]:
# BFS 搜索
search("同济大学","人民广场")

'同济大学->四平路->曲阳路->虹口足球场->西藏北路->中兴路->曲阜路->人民广场'

In [110]:
# 按最短距离搜索
search_by_shortest("同济大学","人民广场")

'同济大学->四平路->邮电新村->海伦路->四川北路->天潼路->南京东路->人民广场'

In [111]:
# 按最少换乘次数搜索
search_by_min_transfers("同济大学","人民广场")

'同济大学->四平路->邮电新村->海伦路->四川北路->天潼路->南京东路->豫园->老西门->新天地->陕西南路->南京西路->人民广场'

In [112]:
# 结合最少换乘次数和最短距离搜索
search_combine("同济大学","人民广场")

'同济大学->四平路->邮电新村->海伦路->四川北路->天潼路->南京东路->人民广场'